In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xlsxwriter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
import tkinter as tk
import customtkinter as ctk
import threading
import time
# pip install beautifulsoup
# pip install requests
# pip install pandas
# pip install xlsxwriter

In [ ]:
base_url="https://kontakt.az"
response = requests.get(base_url)
productList = list()

In [ ]:
def scrape_products(search_value, sku_value):
    page = 1
    print(f"Starting search for '{search_value}' with SKU '{sku_value}'...")
    while page < 20:
        url = f"{base_url}/search/?p={page}&q={search_value}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        productCards = soup.findAll("div", attrs={"class": "prodItem product-item"})
        for productCard in productCards:
            sku = productCard.get("data-sku")
            if sku and sku_value in sku:
                productId = productCard.get("id")
                productName = productCard.find("div", attrs={"class": "prodItem__title"}).text.strip()
                productPriceBox = productCard.find("div", attrs={"class": "prodItem__prices prodItem__prices--active"})
                productImage = productCard.find("img", attrs={"class": "product-image"}).get("src")
                
                originalPrice = productPriceBox.find("i").text if productPriceBox.find("i") else "N/A"
                discountedPrice = productPriceBox.find("b").text if productPriceBox.find("b") else "N/A"
                interestFreeTerm = productCard.find("span").text if productCard.find("span") else "N/A"

                print(f"name: {productName}\nprice: {originalPrice}\ndiscounted price: {discountedPrice}\nimage link: {productImage}\ninterestFreeTerm: {interestFreeTerm}\n")
                productList.append([productId, productName, originalPrice, discountedPrice, productImage])
        
        page += 1
        print(f"Page {page} processed.")
    print("Search completed.")


In [ ]:
def on_search():
    global search_thread
    search_value = searchvalue.get()
    sku_value = data_skuvalue.get()
    if search_value and sku_value:
        search_thread = threading.Thread(target=scrape_products, args=(search_value, sku_value))
        search_thread.start()

def on_cancel():
    stopEvent.set()
    print("Canceling the process...")

app = ctk.CTk()
app.geometry("350x250")
app.title("Search Interface")

searchlabel = ctk.CTkLabel(app, text="Search Value:")
searchlabel.grid(row=0, padx=20, pady=5, sticky="w")
searchvalue = ctk.CTkEntry(app, width=300)
searchvalue.insert(0, "telefon")
searchvalue.grid(row=1, padx=20, pady=5)

data_skulabel = ctk.CTkLabel(app, text="Data Sku Value:")
data_skulabel.grid(row=2, padx=20, pady=5, sticky="w")
data_skuvalue = ctk.CTkEntry(app, width=300)
data_skuvalue.insert(0, "TM-DG-SBP-1105-SM")
data_skuvalue.grid(row=3, padx=20, pady=5)

search_button = ctk.CTkButton(app, text="Search", command=on_search, width=300)
search_button.grid(row=4, column=0, padx=20, pady=5, sticky="w")

cancel_button = ctk.CTkButton(app, text="Cancel", command=on_cancel, width=300)
cancel_button.grid(row=5, column=0, padx=20, pady=5, sticky="w")

stopEvent = threading.Event()

app.mainloop()


In [ ]:
df = pd.DataFrame(productList, columns=["ID", "Məhsulun adı", "Məhsulun endirimsiz qiyməti", "Məhsulun nağd alışa endirimli qiyməti", "Məhsulun şəkil linki"])

In [ ]:
# Bütün datanın excel faylına extract olunması
# Exrteacting all data to excel file
with pd.ExcelWriter("dataframe.xlsx", engine="xlsxwriter") as dataframe:
    df.to_excel(dataframe, sheet_name="sheet1", index=False)
    workbook = dataframe.book
    worksheet = dataframe.sheets["sheet1"]

    for idx, col in enumerate(df.columns):
        # cədvəldəki ən uzun data eninin tapılması
        max_length = max(df[col].astype(str).map(len).max(), len(col)) + 2  # 2 padding dəyəri kimidi
        worksheet.set_column(idx, idx, max_length)  # həmin uzunluğun header uzunluğu kimi təyini

    # header format
    header_format = workbook.add_format({
        'align': 'left',
        'bg_color': '#59788E',
        'italic': True
    })
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)